In [1]:
import sys
import os
import numpy as np
from pathlib import Path
from scipy.stats import pearsonr
root = Path(os.getcwd()).parent.parent
sys.path.append(str(Path(os.getcwd()).parent))
sys.path.append('C:/Users/Yejin Kim/Anaconda3/envs/SolarPowerPrediction/Lib/site-packages')
np.set_printoptions(threshold=sys.maxsize, suppress=True)

In [2]:
from keras.layers import Dense, LSTM, Dropout, Input, RepeatVector, TimeDistributed, Activation, SimpleRNN, GRU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model, Sequential, Model
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import MinMaxScaler
from loader import Power, Weather
from scipy.stats import iqr
from constant import FeatureType
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import argparse
import hashlib
import math

### Setting arguments

In [3]:
parser = argparse.ArgumentParser()
args = parser.parse_args("")

# ====== Model ====== #
args.frame_in = 72
args.frame_out = 24


# ====== Data ====== #
args.years = [2017, 2018, 2019]
args.region = "Jindo"
args.station = 192

# ====== Features ====== #
features = [FeatureType.SUNSHINE,
            FeatureType.GROUND_TEMPERATURE,
            FeatureType.HUMIDITY,
            FeatureType.WIND_SPEED,
            FeatureType.WIND_DIRECTION,
            FeatureType.TEMPERATURE,
            FeatureType.VISIBILITY,
            FeatureType.PRECIPITATION,
            FeatureType.STEAM_PRESSURE,
            FeatureType.DEW_POINT_TEMPERATURE,
            FeatureType.ATMOSPHERIC_PRESSURE]

power = Power(args)
weather = Weather(args, features)

power_data = power.to_dataframe(power.get_data())
weather_data = weather.get_data()

0 missing dates
0 value(s) are not zero
[0, 0, 0, 0, 0, 0, 4, 28, 59, 125, 174, 223, 220, 188, 122, 33, 5, 0, 0, 0, 0, 0, 0]
0 missing dates
0 value(s) are not zero
[0, 0, 0, 0, 0, 0, 3, 21, 63, 123, 180, 233, 243, 198, 131, 31, 3, 0, 0, 0, 0, 0, 0]
0 missing dates
0 value(s) are not zero
[0, 0, 0, 0, 0, 0, 5, 22, 57, 117, 159, 0, 211, 226, 126, 92, 20, 4, 0, 0, 0, 0, 0]
missing dates: []
missing dates: ['2018-01-13 09:00', '2018-01-13 10:00', '2018-01-13 11:00', '2018-01-13 12:00', '2018-01-13 13:00']
missing dates: []


### Set dataset

In [4]:
start = datetime.strptime("%d-01-01 00:00" % args.years[0], "%Y-%m-%d %H:%M")
end = datetime.strptime("%d-12-31 23:00" % args.years[-1], "%Y-%m-%d %H:%M")
duration = (end - start).days + 1

# durations
train_duration = math.floor(duration * 0.6)
val_duration = math.floor(duration * 0.2)
test_duration = math.floor(duration * 0.2)

# days
train_start = start
train_end = train_start + timedelta(days=train_duration - 1)
val_start = train_end + timedelta(days=1)
val_end = val_start + timedelta(days=val_duration - 1)
test_start = val_end + timedelta(days=1)
test_end = test_start + timedelta(days=test_duration - 1)

train_end += timedelta(hours=23)
val_end += timedelta(hours=23)
test_end += timedelta(hours=23)

print("train start date:", str(train_start))
print("train end date:", str(train_end))
print("val start date:", str(val_start))
print("val end date:", str(val_end))
print("test start date:", str(test_start))
print("test end date:", str(test_end))

train start date: 2017-01-01 00:00:00
train end date: 2018-10-19 23:00:00
val start date: 2018-10-20 00:00:00
val end date: 2019-05-26 23:00:00
test start date: 2019-05-27 00:00:00
test end date: 2019-12-31 23:00:00


In [5]:
# split data
train_mask = (power_data.index >= train_start) & (power_data.index <= train_end)
train_power = power_data.loc[train_mask]['power'].to_numpy()
train_weather = weather_data.loc[train_mask]

val_mask = (power_data.index >= val_start) & (power_data.index <= val_end)
val_power = power_data.loc[val_mask]['power'].to_numpy()
val_weather = weather_data.loc[val_mask]

test_mask = (power_data.index >= test_start) & (power_data.index <= test_end)
test_power = power_data.loc[test_mask]['power'].to_numpy()
test_weather = weather_data.loc[test_mask]

# scale data
power_scaler, y_train, y_val, y_test = power.scale(train_power, val_power, test_power)
weather_scalers = []
X_train = []
X_val = []
X_test = []
for feature in weather_data.columns:
    x_train = train_weather[feature].to_numpy()
    x_val = val_weather[feature].to_numpy()
    x_test = test_weather[feature].to_numpy()
    weather_scaler, x_train, x_val, x_test = weather.scale(x_train, x_val, x_test)
    weather_scalers.append(weather_scaler)
    X_train.append(x_train)
    X_val.append(x_val)
    X_test.append(x_test)
    
X_train = np.asarray(X_train)
X_val = np.asarray(X_val)
X_test = np.asarray(X_test)

partition = {'train': [X_train, y_train], 'val': [X_val, y_val], 'test': [X_test, y_test]}

### Set model

In [6]:
# model
def encoder_decoder(partition, args):
    X_train, y_train = partition['train']
    X_val, y_val = partition['val']
    
    model = Sequential()
    model.add(LSTM(256,input_shape=(args.frame_in, len(args.features))))
    model.add(RepeatVector(args.frame_out))
    model.add(LSTM(256, return_sequences=True))
    model.add(TimeDistributed(Dense(256, activation='relu')))
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss='mse', optimizer='rmsprop')
    
    hash_key = hashlib.sha1(str(args).encode()).hexdigest()[:6]
    print("model name: ", hash_key)
    
    model_root_path = os.path.join(root, 'models', hash_key)
    Path(model_root_path).mkdir(parents=True, exist_ok=True)
    checkpoint_path = os.path.join(model_root_path, 'model-{epoch:03d}-{val_loss:03f}.h5')
    model_path = os.path.join(model_root_path, '%s.h5' % 'hash_key')
    
    callback = EarlyStopping(monitor='val_loss', patience=30)
    checkpoint = ModelCheckpoint(checkpoint_path, verbose=1, monitor='val_loss',save_best_only=True)
    
    history = model.fit(X_train, y_train, batch_size=64, epochs=128,
                      validation_data=(X_val, y_val),callbacks=[callback,checkpoint], shuffle=True)
    model.save(model_path)

    return model, history

def predict(partition, model, scaler):
    X_test, y_test = partition['test']
    y_test = y_test.reshape((y_test.shape[0] * y_test.shape[1], y_test.shape[2]))
    y_test = scaler.inverse_transform(y_test)
    
    y_pred = model.predict(X_test)
    y_pred = y_pred.reshape((-1, 1))
    y_pred = scaler.inverse_transform(y_pred)
    
    print(y_test.shape, y_pred.shape)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    max_min = np.max(y_test) - np.min(y_test)
    nrmse = rmse / max_min
    
    print('rmse:', rmse)
    print('nrmse:', nrmse)
    
    return y_test, y_pred, nrmse

### Experiment

In [7]:
def make_x_sample(x_data, feature_len):
    x_data = x_data[0:feature_len]
    x_data = x_data.reshape((x_data.shape[0], x_data.shape[1]))
    x_data = x_data.transpose()
    
    x_list = []
    sample_count = math.floor((x_data.shape[0] + 24 - 96) / 24)
    for i in range(sample_count):
        x = x_data[(i*24):(i*24)+72]
        x_list.append(x)
    x_list = np.asarray(x_list)
    return x_list

def make_y_sample(y_data):
    y_list = []
    sample_count = math.floor((y_data.shape[0] + 24 - 96) / 24)
    for i in range(sample_count):
        y = y_data[(i*24)+72:(i*24)+72+24]
        y_list.append(y)
    y_list = np.asarray(y_list)
    return y_list

y_pred_list = []
nrmse_list = []
for i in range(len(features)):
    args_feature = [features[j].value for j in range(i+1)]
    print(args_feature)
    setattr(args, 'features', args_feature)
    
    X_train, y_train = partition['train']
    X_val, y_val = partition['val']
    X_test, y_test = partition['test']
    
    X_train = make_x_sample(X_train, i+1)
    X_val = make_x_sample(X_val, i+1)
    X_test = make_x_sample(X_test, i+1)
    
    y_train = make_y_sample(y_train)
    y_val = make_y_sample(y_val)
    y_test = make_y_sample(y_test)
    
    print(X_train.shape, X_val.shape, X_test.shape)
    print(y_train.shape, y_val.shape, y_test.shape)
    
    new_partition = {'train': [X_train, y_train], 'val': [X_val, y_val], 'test': [X_test, y_test]}
    
    model, history = encoder_decoder(new_partition, args)
    y_test, y_pred, nrmse = predict(new_partition, model, power_scaler)
    y_pred_list.append(y_pred)
    nrmse_list.append(nrmse)

['일조(hr)']
(654, 72, 1) (216, 72, 1) (216, 72, 1)
(654, 24, 1) (216, 24, 1) (216, 24, 1)
model name:  f183d3
Epoch 1/128
11/11 [==============================] - ETA: 0s - loss: 0.0061
Epoch 00001: val_loss improved from inf to 0.00139, saving model to C:\Users\Yejin Kim\Yejining\Github\2020 DistributedResource\SolarPowerPrediction\models\f183d3\model-001-0.001393.h5
11/11 [==============================] - 3s 263ms/step - loss: 0.0061 - val_loss: 0.0014
Epoch 2/128
11/11 [==============================] - ETA: 0s - loss: 0.0016
Epoch 00002: val_loss improved from 0.00139 to 0.00139, saving model to C:\Users\Yejin Kim\Yejining\Github\2020 DistributedResource\SolarPowerPrediction\models\f183d3\model-002-0.001393.h5
11/11 [==============================] - 2s 206ms/step - loss: 0.0016 - val_loss: 0.0014
Epoch 3/128
11/11 [==============================] - ETA: 0s - loss: 0.0016
Epoch 00003: val_loss improved from 0.00139 to 0.00122, saving model to C:\Users\Yejin Kim\Yejining\Github\2020

Epoch 32/128
11/11 [==============================] - ETA: 0s - loss: 5.4746e-04
Epoch 00032: val_loss did not improve from 0.00031
11/11 [==============================] - 2s 192ms/step - loss: 5.4746e-04 - val_loss: 3.1612e-04
Epoch 33/128
11/11 [==============================] - ETA: 0s - loss: 4.9382e-04
Epoch 00033: val_loss did not improve from 0.00031
11/11 [==============================] - 2s 184ms/step - loss: 4.9382e-04 - val_loss: 4.9710e-04
Epoch 34/128
11/11 [==============================] - ETA: 0s - loss: 4.9318e-04
Epoch 00034: val_loss did not improve from 0.00031
11/11 [==============================] - 2s 185ms/step - loss: 4.9318e-04 - val_loss: 3.5879e-04
Epoch 35/128
11/11 [==============================] - ETA: 0s - loss: 4.7848e-04
Epoch 00035: val_loss did not improve from 0.00031
11/11 [==============================] - 2s 187ms/step - loss: 4.7848e-04 - val_loss: 4.3141e-04
Epoch 36/128
11/11 [==============================] - ETA: 0s - loss: 4.9603e-04
Epo

11/11 [==============================] - ETA: 0s - loss: 4.4056e-04
Epoch 00100: val_loss did not improve from 0.00030
11/11 [==============================] - 2s 185ms/step - loss: 4.4056e-04 - val_loss: 4.9765e-04
Epoch 101/128
11/11 [==============================] - ETA: 0s - loss: 4.3189e-04
Epoch 00101: val_loss did not improve from 0.00030
11/11 [==============================] - 2s 183ms/step - loss: 4.3189e-04 - val_loss: 3.0027e-04
Epoch 102/128
11/11 [==============================] - ETA: 0s - loss: 4.1147e-04
Epoch 00102: val_loss did not improve from 0.00030
11/11 [==============================] - 2s 178ms/step - loss: 4.1147e-04 - val_loss: 2.9973e-04
Epoch 103/128
11/11 [==============================] - ETA: 0s - loss: 4.6122e-04
Epoch 00103: val_loss did not improve from 0.00030
11/11 [==============================] - 2s 188ms/step - loss: 4.6122e-04 - val_loss: 4.3478e-04
Epoch 104/128
11/11 [==============================] - ETA: 0s - loss: 4.2196e-04
Epoch 00104:

Epoch 10/128
11/11 [==============================] - ETA: 0s - loss: 6.8445e-04
Epoch 00010: val_loss did not improve from 0.00047
11/11 [==============================] - 2s 194ms/step - loss: 6.8445e-04 - val_loss: 6.2710e-04
Epoch 11/128
11/11 [==============================] - ETA: 0s - loss: 7.6306e-04
Epoch 00011: val_loss did not improve from 0.00047
11/11 [==============================] - 2s 199ms/step - loss: 7.6306e-04 - val_loss: 7.7488e-04
Epoch 12/128
11/11 [==============================] - ETA: 0s - loss: 5.7309e-04
Epoch 00012: val_loss improved from 0.00047 to 0.00037, saving model to C:\Users\Yejin Kim\Yejining\Github\2020 DistributedResource\SolarPowerPrediction\models\bd7bbc\model-012-0.000366.h5
11/11 [==============================] - 2s 205ms/step - loss: 5.7309e-04 - val_loss: 3.6613e-04
Epoch 13/128
11/11 [==============================] - ETA: 0s - loss: 0.0010  
Epoch 00013: val_loss did not improve from 0.00037
11/11 [==============================] - 2s 1

Epoch 77/128
11/11 [==============================] - ETA: 0s - loss: 4.2974e-04
Epoch 00077: val_loss did not improve from 0.00028
11/11 [==============================] - 2s 193ms/step - loss: 4.2974e-04 - val_loss: 3.1127e-04
Epoch 78/128
11/11 [==============================] - ETA: 0s - loss: 4.0495e-04
Epoch 00078: val_loss did not improve from 0.00028
11/11 [==============================] - 2s 215ms/step - loss: 4.0495e-04 - val_loss: 5.1056e-04
Epoch 79/128
11/11 [==============================] - ETA: 0s - loss: 4.2874e-04
Epoch 00079: val_loss did not improve from 0.00028
11/11 [==============================] - 2s 194ms/step - loss: 4.2874e-04 - val_loss: 3.6737e-04
Epoch 80/128
11/11 [==============================] - ETA: 0s - loss: 4.2698e-04
Epoch 00080: val_loss did not improve from 0.00028
11/11 [==============================] - 2s 208ms/step - loss: 4.2698e-04 - val_loss: 2.9119e-04
Epoch 81/128
11/11 [==============================] - ETA: 0s - loss: 4.1741e-04
Epo

...

Epoch 31/128
11/11 [==============================] - ETA: 0s - loss: 5.1815e-04
Epoch 00031: val_loss did not improve from 0.00029
11/11 [==============================] - 3s 267ms/step - loss: 5.1815e-04 - val_loss: 3.0241e-04
Epoch 32/128
11/11 [==============================] - ETA: 0s - loss: 5.0485e-04
Epoch 00032: val_loss did not improve from 0.00029
11/11 [==============================] - 3s 301ms/step - loss: 5.0485e-04 - val_loss: 5.0559e-04
Epoch 33/128
11/11 [==============================] - ETA: 0s - loss: 5.1893e-04
Epoch 00033: val_loss did not improve from 0.00029
11/11 [==============================] - 3s 268ms/step - loss: 5.1893e-04 - val_loss: 3.1312e-04
Epoch 34/128
11/11 [==============================] - ETA: 0s - loss: 4.7723e-04
Epoch 00034: val_loss did not improve from 0.00029
11/11 [==============================] - 3s 266ms/step - loss: 4.7723e-04 - val_loss: 4.6413e-04
Epoch 35/128
11/11 [==============================] - ETA: 0s - loss: 4.9734e-04
Epo

Epoch 17/128
11/11 [==============================] - ETA: 0s - loss: 7.9959e-04
Epoch 00017: val_loss improved from 0.00063 to 0.00031, saving model to C:\Users\Yejin Kim\Yejining\Github\2020 DistributedResource\SolarPowerPrediction\models\70bcea\model-017-0.000311.h5
11/11 [==============================] - 3s 286ms/step - loss: 7.9959e-04 - val_loss: 3.1064e-04
Epoch 18/128
11/11 [==============================] - ETA: 0s - loss: 5.3043e-04
Epoch 00018: val_loss did not improve from 0.00031
11/11 [==============================] - 3s 291ms/step - loss: 5.3043e-04 - val_loss: 0.0012
Epoch 19/128
11/11 [==============================] - ETA: 0s - loss: 8.3838e-04
Epoch 00019: val_loss did not improve from 0.00031
11/11 [==============================] - 3s 285ms/step - loss: 8.3838e-04 - val_loss: 4.6325e-04
Epoch 20/128
11/11 [==============================] - ETA: 0s - loss: 5.6051e-04
Epoch 00020: val_loss did not improve from 0.00031
11/11 [==============================] - 3s 289

Epoch 51/128
11/11 [==============================] - ETA: 0s - loss: 4.5252e-04
Epoch 00051: val_loss did not improve from 0.00027
11/11 [==============================] - 3s 278ms/step - loss: 4.5252e-04 - val_loss: 3.5926e-04
Epoch 52/128
11/11 [==============================] - ETA: 0s - loss: 4.3697e-04
Epoch 00052: val_loss did not improve from 0.00027
11/11 [==============================] - 3s 302ms/step - loss: 4.3697e-04 - val_loss: 3.3231e-04
Epoch 53/128
11/11 [==============================] - ETA: 0s - loss: 4.3451e-04
Epoch 00053: val_loss did not improve from 0.00027
11/11 [==============================] - 3s 295ms/step - loss: 4.3451e-04 - val_loss: 5.4348e-04
Epoch 54/128
11/11 [==============================] - ETA: 0s - loss: 4.3510e-04
Epoch 00054: val_loss did not improve from 0.00027
11/11 [==============================] - 3s 308ms/step - loss: 4.3510e-04 - val_loss: 2.8520e-04
Epoch 55/128
11/11 [==============================] - ETA: 0s - loss: 4.6434e-04
Epo

Epoch 6/128
11/11 [==============================] - ETA: 0s - loss: 0.0027
Epoch 00006: val_loss did not improve from 0.00129
11/11 [==============================] - 3s 262ms/step - loss: 0.0027 - val_loss: 0.0086
Epoch 7/128
11/11 [==============================] - ETA: 0s - loss: 0.0052
Epoch 00007: val_loss improved from 0.00129 to 0.00128, saving model to C:\Users\Yejin Kim\Yejining\Github\2020 DistributedResource\SolarPowerPrediction\models\6d9e6e\model-007-0.001281.h5
11/11 [==============================] - 3s 295ms/step - loss: 0.0052 - val_loss: 0.0013
Epoch 8/128
11/11 [==============================] - ETA: 0s - loss: 0.0017
Epoch 00008: val_loss did not improve from 0.00128
11/11 [==============================] - 3s 289ms/step - loss: 0.0017 - val_loss: 0.0014
Epoch 9/128
11/11 [==============================] - ETA: 0s - loss: 0.0016
Epoch 00009: val_loss improved from 0.00128 to 0.00121, saving model to C:\Users\Yejin Kim\Yejining\Github\2020 DistributedResource\SolarP

11/11 [==============================] - ETA: 0s - loss: 4.0549e-04
Epoch 00072: val_loss did not improve from 0.00027
11/11 [==============================] - 3s 271ms/step - loss: 4.0549e-04 - val_loss: 3.2209e-04
Epoch 73/128
11/11 [==============================] - ETA: 0s - loss: 3.9772e-04
Epoch 00073: val_loss did not improve from 0.00027
11/11 [==============================] - 3s 250ms/step - loss: 3.9772e-04 - val_loss: 3.7497e-04
Epoch 74/128
11/11 [==============================] - ETA: 0s - loss: 4.0954e-04
Epoch 00074: val_loss improved from 0.00027 to 0.00026, saving model to C:\Users\Yejin Kim\Yejining\Github\2020 DistributedResource\SolarPowerPrediction\models\6d9e6e\model-074-0.000264.h5
11/11 [==============================] - 3s 249ms/step - loss: 4.0954e-04 - val_loss: 2.6406e-04
Epoch 75/128
11/11 [==============================] - ETA: 0s - loss: 3.8544e-04
Epoch 00075: val_loss did not improve from 0.00026
11/11 [==============================] - 3s 272ms/step -

In [8]:
df = pd.DataFrame()
for i in range(len(features)):
    y_pred_list[i] = y_pred_list[i].reshape((y_pred_list[i].shape[0]))
    df['%dth model' % (i + 1)] = y_pred_list[i].tolist()
y_test = y_test.reshape((y_test.shape[0]))
df['y_test'] = y_test.tolist()

full_idx = pd.date_range(start=test_start+timedelta(days=3), end=test_end, freq='H')
full_idx = full_idx[:y_test.shape[0]]
df['time'] = full_idx
df = df.set_index('time')

result_path = os.path.join(root, 'results')
file_count = len([name for name in os.listdir(result_path) if os.path.isfile(os.path.join(result_path, name))])
result_name = os.path.join(result_path, '%dth trial.csv' % (file_count + 1))
df.to_csv(result_name)

In [9]:
nrmse_list

[0.11151867972987871,
 0.11509012115223223,
 0.11641467967162801,
 0.12897109602188522,
 0.1397952965506016,
 0.1383865712732853,
 0.11734956360687743,
 0.1243463638367827,
 0.11730466511904872,
 0.13446502138684582,
 0.10899599780938637]